In [59]:
#Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

# set the max columns to none
pd.set_option('display.max_columns', None)



In [60]:
## Trying another approach to get the model

from typing import Dict
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
#import dectvectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import MaxAbsScaler
df = pd.read_csv('../datasets/customer_churn_dataset-master.csv')

#Drop the columns we don't need
df.drop(['CustomerID'], axis = 1, inplace = True)
df = df.dropna()
#df['Attrition'] = df['Attrition'].map({'Yes':1, 'No':0})
#df['Over18'] = df['Over18'].map({'Yes':1, 'No':0})
#df['OverTime'] = df['OverTime'].map({'Yes':1, 'No':0})
#Categorical data 
categorical = ['Gender', 'Subscription Type', 'Contract Length']
#Numerical data
numerical = ['Age', 'Tenure', 'Usage Frequency',	'Support Calls', 'Payment Delay', 'Total Spend', 'Last Interaction']

## Divide the data into train and test
df_train_all, df_test =train_test_split(df, test_size = 0.2, random_state = 0)
##Obtain y values
y_train_all = df_train_all['Churn'].astype(int).values
y_test = df_test['Churn'].astype(int).values

## Training model 
df_train, df_val = train_test_split(df_train_all, test_size = 0.20, random_state = 0)
y_train = df_train['Churn'].astype(int).values
y_val = df_val['Churn'].astype(int).values
## Use DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient = 'records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)


In [61]:
## Use LogisticRegression() to train the model
logreg = LogisticRegression()
## Applying MaxAbsScaler() to the data
scaler = MaxAbsScaler()
X_train = scaler.fit_transform(X_train)
logreg.fit(X_train, y_train)

LogisticRegression()

In [62]:
## Transform validation set to dicts
val_dicts = df_val[categorical + numerical].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)
X_val = scaler.transform(X_val)

##Ap

## Predict on validation set
y_pred = logreg.predict_proba(X_val)[:,1]

In [63]:
## Roc Auc Score
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, y_pred)

0.9588276871068936

In [64]:
## Let's make the pipeline
from sklearn.pipeline import make_pipeline

Pipeline = make_pipeline(
    DictVectorizer(),
    MaxAbsScaler(),
    LogisticRegression(),

)

## Fit the pipeline
Pipeline.fit(train_dicts, y_train)

Pipeline(steps=[('dictvectorizer', DictVectorizer()),
                ('maxabsscaler', MaxAbsScaler()),
                ('logisticregression', LogisticRegression())])

In [65]:
#Test pipeline
y_pred = Pipeline.predict_proba(val_dicts)[:,1]

## Evaluate the model
roc_auc_score(y_val, y_pred)

0.9588276871068936

In [66]:
import pickle
with open ('../models/pipeline.bin', 'wb' ) as f_out: 
    pickle.dump(Pipeline, f_out)

In [67]:
df_val

,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
148888,21.0,Male,31.0,15.0,2.0,9.0,Standard,Annual,535.00,28.0,0.0
199391,39.0,Female,48.0,18.0,9.0,13.0,Basic,Quarterly,456.73,5.0,1.0
271920,44.0,Male,22.0,17.0,4.0,16.0,Basic,Annual,813.50,22.0,0.0
408341,29.0,Male,31.0,14.0,0.0,10.0,Basic,Annual,853.26,14.0,0.0
91211,25.0,Female,18.0,1.0,1.0,22.0,Standard,Quarterly,598.00,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
404502,50.0,Female,11.0,29.0,0.0,3.0,Basic,Annual,532.05,2.0,0.0
159412,47.0,Male,4.0,29.0,6.0,29.0,Premium,Quarterly,117.00,21.0,1.0
396594,41.0,Male,46.0,1.0,2.0,9.0,Premium,Annual,747.45,8.0,0.0
403229,47.0,Male,47.0,27.0,3.0,16.0,Basic,Annual,793.66,21.0,0.0
